In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df = pd.read_csv('./data/processed/df.csv')
df

,review,score,sentiment
0,판타지를 넘어 영화 역사에 남을 명작이다 내가 이걸 왜 극장에서 못봤을까 폰으로 봤...,10.0,1.0
1,이집트여행하는느낌의 영화,8.0,1.0
2,목포 연설 장면은 넋을 잃고 보게 된다,10.0,1.0
3,그저 그렇네요 뻔한 반전과 결말,4.0,0.0
4,더 배트맨 조커처럼 몰입감이 있으면 추천 반대로 지루하면 비추천,1.0,0.0
...,...,...,...
9326,이해할 수 없는 남자와 이해해주는 여자,7.0,0.0
9327,배우들 연기력이 장난아님 역시 최민식 배우는 클라스가 다름,10.0,1.0
9328,그래도 최근에 나왔던 영화들에 비해 내용이 되게 흥미진진했음 리암 니슨이 덜뛰는 대...,10.0,1.0
9329,누구나 시작은 어설픈 법 그래서 더 매력적인 히어로,10.0,1.0


In [6]:
stopwords = pd.read_table('./data/stopwords.txt')
#['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords

,word
0,가
1,가까스로
2,가령
3,각
4,각각
...,...
593,휘익
594,휴
595,흐흐
596,흥


In [7]:
stopwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    598 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


In [8]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [17]:
X_train = []
for sentence in tqdm(df['review'][:7465]):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

100%|██████████| 7465/7465 [00:47<00:00, 156.40it/s]


In [14]:
df['review'][:7466]

0       판타지를 넘어 영화 역사에 남을 명작이다 내가 이걸 왜 극장에서 못봤을까 폰으로 봤...
1                                           이집트여행하는느낌의 영화
2                                   목포 연설 장면은 넋을 잃고 보게 된다
3                                       그저 그렇네요 뻔한 반전과 결말
4                     더 배트맨 조커처럼 몰입감이 있으면 추천 반대로 지루하면 비추천
                              ...                        
7461    잠이 안 와서 또 봤는데 이번이 번째인데 볼 때 마다 또 소름 돋고 영화가 갖는 의...
7462         킬링타임으로 추천특히 두 남여샘 연기 감초중 감초 박정민 배우님 완전 반가웠어요
7463    잔인하거나 자극적인걸 극혐해서 영화 보는것을 좋아하지 않아서 안보다가 무섭지 않은 ...
7464                    이뻐져도골비면 오래못간다라는 메세지캐릭터가 멍청해서 짜증났음
7465    이게 진짜 재밌나저예산 급 영화왠만한 저평가 영화도 재밌게 보는 편인데진짜 재미없음...
Name: review, Length: 7466, dtype: object

In [10]:
print(X_train[:3])

[['판타지', '를', '넘다', '영화', '역사', '에', '남', '을', '명작', '이다', '내', '가', '이', '걸', '왜', '극장', '에서', '못', '보다', '폰', '으로', '보다', '때', '의', '감동', '과는', '비교', '가', '안되다'], ['이집트', '여행', '하', '는', '느낌', '의', '영화'], ['목포', '연설', '장면', '은', '넋', '을', '잃다', '보다', '되다']]


In [19]:
X_test = []
for sentence in tqdm(df['review'][7465:]):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

100%|██████████| 1866/1866 [00:09<00:00, 204.75it/s]
